In [39]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [40]:
dfs_submissions, dfs_comments = [], []

for file in os.listdir('data'):
    file_size_bytes = round(os.path.getsize(f"data/{file}") / 1024**2, 4)
    df = pd.read_csv(f"data/{file}", low_memory=False)

    print(f"File: {file}, Size: {file_size_bytes} Mb, Rows: {df.shape[0]}, Columns: {df.shape[1]}")

    if 'RS' in file:
        dfs_submissions.append(df)

    if 'RC' in file:
        dfs_comments.append(df)

File: RS_2025-04.csv, Size: 81.1134 Mb, Rows: 37932, Columns: 118
File: RS_2025-05.csv, Size: 68.6732 Mb, Rows: 32826, Columns: 118
File: RS_2025-02.csv, Size: 88.3056 Mb, Rows: 43877, Columns: 118
File: RS_2025-03.csv, Size: 97.044 Mb, Rows: 46129, Columns: 118
File: RC_2025-04.csv, Size: 963.5798 Mb, Rows: 1378706, Columns: 74
File: RC_2025-05.csv, Size: 759.3355 Mb, Rows: 1069281, Columns: 74
File: RC_2025-02.csv, Size: 988.9622 Mb, Rows: 1428380, Columns: 74
File: RC_2025-03.csv, Size: 1163.4925 Mb, Rows: 1680448, Columns: 74


In [41]:
df_comments = pd.concat(dfs_comments, axis=0)
df_submissions = pd.concat(dfs_submissions, axis=0)

In [42]:
df_comments

,_meta,all_awardings,approved_at_utc,approved_by,archived,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,...,top_awarded_type,total_awards_received,treatment_tags,unrepliable_reason,updated_on,ups,user_reports,author_cakeday,editable,media_metadata
0,{'retrieved_2nd_on': 1743595218},[],NaN,NaN,False,NaN,squirrel9000,NaN,NaN,[],...,NaN,0,[],NaN,1.743466e+09,40,[],NaN,NaN,NaN
1,{'retrieved_2nd_on': 1743595218},[],NaN,NaN,False,NaN,TessaAlGul,NaN,NaN,[],...,NaN,0,[],NaN,1.743466e+09,3,[],NaN,NaN,NaN
2,{'retrieved_2nd_on': 1743595231},[],NaN,NaN,False,NaN,bolonomadic,NaN,NaN,[],...,NaN,0,[],NaN,1.743466e+09,17,[],NaN,NaN,NaN
3,{'retrieved_2nd_on': 1743595231},[],NaN,NaN,False,NaN,[deleted],NaN,NaN,NaN,...,NaN,0,[],NaN,1.743466e+09,1,[],NaN,NaN,NaN
4,{'retrieved_2nd_on': 1743595234},[],NaN,NaN,False,NaN,amontpetit,NaN,NaN,"[{'e': 'text', 't': 'Hamilton'}]",...,NaN,0,[],NaN,1.743466e+09,3,[],NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680443,{'retrieved_2nd_on': 1743595202},[],NaN,NaN,False,NaN,Fringe-Farmer,NaN,NaN,[],...,NaN,0,[],NaN,1.743466e+09,3,[],NaN,NaN,NaN
1680444,{'retrieved_2nd_on': 1743595202},[],NaN,NaN,False,NaN,thedoodely,#dadada,NaN,[],...,NaN,0,[],NaN,1.743466e+09,2,[],NaN,NaN,NaN
1680445,{'retrieved_2nd_on': 1743595205},[],NaN,NaN,False,NaN,SnowFlakeUsername2,#dadada,SK,"[{'e': 'text', 't': 'Saskatchewan '}, {'a': ':...",...,NaN,0,[],NaN,1.743466e+09,5,[],NaN,NaN,NaN
1680446,{'retrieved_2nd_on': 1743595210},[],NaN,NaN,False,NaN,jarretwithonet,NaN,NaN,[],...,NaN,0,[],NaN,1.743466e+09,8,[],NaN,NaN,NaN


In [43]:
df_submissions

,_meta,all_awardings,allow_live_comments,approved_at_utc,approved_by,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,...,gallery_data,is_gallery,media_metadata,post_hint,preview,previous_selftext,crosspost_parent,crosspost_parent_list,author_cakeday,poll_data
0,{'retrieved_2nd_on': 1743595282},[],False,NaN,NaN,False,KauztiK,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'removal_type': 'moderator', 'retrieved_2nd_o...",[],False,NaN,NaN,False,Emmerson_Brando,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{'retrieved_2nd_on': 1743595362},[],False,NaN,NaN,False,Outrageous-Fly-902,NaN,NaN,[],...,"{'items': [{'caption': '', 'id': 634323866, 'm...",True,"{'3gp68orr64se1': {'e': 'Image', 'id': '3gp68o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'retrieved_2nd_on': 1743595363},[],False,NaN,NaN,False,KauztiK,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'removal_type': 'moderator', 'retrieved_2nd_o...",[],False,NaN,NaN,False,MrZapfe,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46124,{'retrieved_2nd_on': 1743595035},[],False,NaN,NaN,False,OkTangerine7,NaN,NaN,[],...,NaN,NaN,NaN,self,"{'enabled': False, 'images': [{'id': 'jq2qXBaL...",NaN,NaN,NaN,NaN,NaN
46125,{'retrieved_2nd_on': 1743595086},[],False,NaN,NaN,False,RPG_Vancouver,NaN,NaN,[],...,NaN,NaN,NaN,link,"{'enabled': False, 'images': [{'id': '05WCmnrR...",NaN,NaN,NaN,NaN,NaN
46126,{'retrieved_2nd_on': 1743595121},[],False,NaN,NaN,False,LeGrandLucifer,NaN,NaN,[],...,NaN,NaN,NaN,link,"{'enabled': False, 'images': [{'id': '1_dDuSIR...",NaN,NaN,NaN,NaN,NaN
46127,{'retrieved_2nd_on': 1743595164},[],False,NaN,NaN,False,n0tso0bvious,NaN,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_comments.shape, df_submissions.shape

((5556815, 74), (160764, 118))

In [45]:
df_comments.dtypes.value_counts()

object     38
float64    16
bool       12
int64       8
Name: count, dtype: int64

In [46]:
df_submissions.dtypes.value_counts()

object     58
bool       28
float64    22
int64      10
Name: count, dtype: int64